In [8]:
from config import *
import tweepy
import psycopg2
import getpass
import json
from datetime import datetime as dt

p = 'dataisgreat'

# p = getpass.getpass(prompt="Password: ")
# conn = psycopg2.connect(user = "postgres",
#                                   password = f"{p}",
#                                   host = "localhost",
#                                   port = "5432",
#                                   database = "twitter_db")

conn = psycopg2.connect(user = "postgres",
                                  password = f"{p}",
                                  host = "127.0.0.1",
                                  port = "3306",
                                  database = "postgres")

cursor = conn.cursor()

# cursor.execute("select * from twitter_feed")

# fetch = cursor.fetchone()
# print(fetch)
# rds_connection_string = f"postgres:{p}@localhost:5432/twitter_db"
# engine = create_engine(f'postgresql://{rds_connection_string}')

In [9]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

my_list = []
class MyStreamListener(tweepy.StreamListener):
    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0
    
    def on_status(self, status):
        collection = []
        print(status.text)
        #Don't know why it doesnt print out coordinates
        print(status.coordinates)
        self.num_tweets += 1
        if self.num_tweets < 20:
            collection.append(status)
            return True
        else:
            return False
        
    def on_data(self, data):
        print("On Data")
        all_data = json.loads(data)
#         print(json.dumps(all_data, indent=4))
        date = all_data['created_at']
        place = all_data['user']['location']
        tweet = all_data["text"].replace('"',"")
        username = all_data["user"]["screen_name"]
        metadata = all_data['user']["favourites_count"]
        print(date)
        print((place,username,tweet))
        self.num_tweets += 1
        if self.num_tweets < 500:
            cursor.execute('INSERT INTO twitter_feed (date, user_location, user_id, text, metadata) \
            VALUES (%s,%s,%s,%s,%s);', (dt.strptime(date,'%a %b %d %H:%M:%S %z %Y').strftime('%Y-%m-%d %H:%M:%S'),place,tweet,username,metadata))
            
            cursor.execute('INSERT INTO tweets (data) VALUES (%s);', (json.dumps(all_data),))
            conn.commit()
            
            print(self.num_tweets)
            print((username,tweet))

            return True
        else:
            return False
    
        


myStreamListener = MyStreamListener()

myStream = tweepy.Stream(auth=api.auth, listener=myStreamListener)


myStream.filter(track=['election2020'], is_async=True)

Authentication OK
On Data
Fri Feb 21 05:53:11 +0000 2020
(None, 'SoniaHi27303526', 'RT @BbyDmplng403: @LouiseLowrey There are so many hilarious comments in You Tube @PatriotsSoapbox chat on @_saltycrystal show tonight.')
1
('SoniaHi27303526', 'RT @BbyDmplng403: @LouiseLowrey There are so many hilarious comments in You Tube @PatriotsSoapbox chat on @_saltycrystal show tonight.')
On Data
Fri Feb 21 05:53:15 +0000 2020
(None, 'SoniaHi27303526', 'RT @DanRodimer: Democrat frontrunner Michael Bloomberg says he wants to make the whole country behave while dressed as Mary Poppins.\n\nWel…')
2
('SoniaHi27303526', 'RT @DanRodimer: Democrat frontrunner Michael Bloomberg says he wants to make the whole country behave while dressed as Mary Poppins.\n\nWel…')
On Data
Fri Feb 21 05:53:15 +0000 2020
('Germany', 'julianfperez', 'The latest COACH! https://t.co/Sz1NOFbZmK #election2020 #misleadingtweets')
3
('julianfperez', 'The latest COACH! https://t.co/Sz1NOFbZmK #election2020 #misleadingtweets')
On D

In [9]:
date ='Fri Feb 21 03:36:01 +0000 2020'

In [10]:
date

'Fri Feb 21 03:36:01 +0000 2020'

On Data
Fri Feb 21 03:37:14 +0000 2020
('🌍 🇺🇸 ✊🏾', 'Modmother', '#NoVote4Zionists #BDS #Election2020 #Reparations NOT #Incarceration! #Blackwash is Illogical: #BlackLivesMatter bec… https://t.co/4GbnZgkA9u')
INSERT INTO twitter_feed (date, user_location, user_id, text, metadata) VALUES (Fri Feb 21 03:37:14 +0000 2020,"🌍 🇺🇸 ✊🏾","#NoVote4Zionists #BDS #Election2020 #Reparations NOT #Incarceration! #Blackwash is Illogical: #BlackLivesMatter bec… https://t.co/4GbnZgkA9u",Modmother,33506);
('Modmother', '#NoVote4Zionists #BDS #Election2020 #Reparations NOT #Incarceration! #Blackwash is Illogical: #BlackLivesMatter bec… https://t.co/4GbnZgkA9u')
On Data
{
    "created_at": "Fri Feb 21 03:37:14 +0000 2020",
    "id": 1230697963313172480,
    "id_str": "1230697963313172480",
    "text": "#NoVote4Zionists #BDS #Election2020 #Reparations NOT #Incarceration! #Blackwash is Illogical: #BlackLivesMatter bec\u2026 https://t.co/4GbnZgkA9u",
    "display_text_range": [
        0,
        140
    ],
   

On DataOn Data

Fri Feb 21 03:37:15 +0000 2020
('#MAGA #KAG TRUMP 2020 COUNTRY!', 'RichardVeii', "The Democrats are still trying to figure out how to use the new fangled iPad thingy's!\n\nHere We Go Again: Dems Won'… https://t.co/tooLafQHIH")
{
    "created_at": "Fri Feb 21 03:37:15 +0000 2020",
    "id": 1230697967205507073,
    "id_str": "1230697967205507073",
    "text": "The Democrats are still trying to figure out how to use the new fangled iPad thingy's!\n\nHere We Go Again: Dems Won'\u2026 https://t.co/tooLafQHIH",
    "source": "<a href=\"http://twitter.com\" rel=\"nofollow\">Twitter Web Client</a>",
    "truncated": true,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_str": null,
    "in_reply_to_screen_name": null,
    "user": {
        "id": 565078449,
        "id_str": "565078449",
        "name": "Rich Hannigan",
        "screen_name": "RichardVeii",
        "location": "#MAGA #KAG TRUMP 2

In [15]:
type(dt.strptime(date,'%a %b %d %H:%M:%S %z %Y').strftime('%Y-%m-%d %H:%M:%S'))

str

In [11]:
cursor.execute("select * from tweets")

fetch = cursor.fetchone()
print(json.dumps(fetch,indent=4))

[
    1,
    {
        "id": 1230732175042961408,
        "geo": null,
        "lang": "en",
        "text": "RT @BbyDmplng403: @LouiseLowrey There are so many hilarious comments in You Tube @PatriotsSoapbox chat on @_saltycrystal show tonight.",
        "user": {
            "id": 1080974939446861825,
            "url": null,
            "lang": null,
            "name": "Sonia Hill",
            "id_str": "1080974939446861825",
            "location": null,
            "verified": false,
            "following": null,
            "protected": false,
            "time_zone": null,
            "created_at": "Thu Jan 03 23:51:23 +0000 2019",
            "utc_offset": null,
            "description": null,
            "geo_enabled": false,
            "screen_name": "SoniaHi27303526",
            "listed_count": 0,
            "friends_count": 1034,
            "is_translator": false,
            "notifications": null,
            "statuses_count": 23058,
            "default_profile": t

In [12]:
cursor.execute("select * from tweets")

fetch = cursor.fetchone()
print(json.dumps(fetch,indent=4))

[
    1,
    {
        "id": 1230732175042961408,
        "geo": null,
        "lang": "en",
        "text": "RT @BbyDmplng403: @LouiseLowrey There are so many hilarious comments in You Tube @PatriotsSoapbox chat on @_saltycrystal show tonight.",
        "user": {
            "id": 1080974939446861825,
            "url": null,
            "lang": null,
            "name": "Sonia Hill",
            "id_str": "1080974939446861825",
            "location": null,
            "verified": false,
            "following": null,
            "protected": false,
            "time_zone": null,
            "created_at": "Thu Jan 03 23:51:23 +0000 2019",
            "utc_offset": null,
            "description": null,
            "geo_enabled": false,
            "screen_name": "SoniaHi27303526",
            "listed_count": 0,
            "friends_count": 1034,
            "is_translator": false,
            "notifications": null,
            "statuses_count": 23058,
            "default_profile": t

On Data
Fri Feb 21 05:56:09 +0000 2020
('South Florida', 'austinm419', "RT @Ellars: It's $3 Thursday! I'm asking for everyone to help that is able to donate $3 to my campaign for Congress. Join us and help put p…")
9
('austinm419', "RT @Ellars: It's $3 Thursday! I'm asking for everyone to help that is able to donate $3 to my campaign for Congress. Join us and help put p…")


In [13]:
type(fetch)

tuple

On Data
Fri Feb 21 05:57:38 +0000 2020
('Washington D.C.', 'RaceForPresNews', '2020 Presidential Election News Update🗞:\n2020 Democratic contenders plead for cash after draining their accounts… https://t.co/1D9EtOncGO')
10
('RaceForPresNews', '2020 Presidential Election News Update🗞:\n2020 Democratic contenders plead for cash after draining their accounts… https://t.co/1D9EtOncGO')
On Data
Fri Feb 21 05:57:48 +0000 2020
(None, 'MillerCrime', 'RT @MillerCrime: #Bloomberg is buying the Dem nomination *because* many resisters won’t support him👀\n\n‼️His goal is to keep the #GOP in pow…')
11
('MillerCrime', 'RT @MillerCrime: #Bloomberg is buying the Dem nomination *because* many resisters won’t support him👀\n\n‼️His goal is to keep the #GOP in pow…')
On Data
Fri Feb 21 05:58:19 +0000 2020
(None, 'HighClassWhite3', 'I wonder just how long it will take for some of these #grassroots organizations to become the new #superPACs if the… https://t.co/1Qc41cyqf3')
12
('HighClassWhite3', 'I wonder j